In [1]:
from py2neo import Graph, Node, Relationship, cypher
import pandas as pd
import numpy

In [2]:
newLinksFile = 'mgmt1.csv'
graph = Graph()

In [3]:
newLinks = pd.read_csv(newLinksFile, delimiter=',')
print(newLinks.columns)

Index(['din', 'cin', 'startdate', 'enddate'], dtype='object')


In [4]:
newLinks.head()

,din,cin,startdate,enddate
0,00006755,U28910PN2007PTC131007,2007-11-20,-
1,02559409,U74900WB2014PTC201980,2016-11-11,-
2,01645731,U74899DL1998PLC093329,1998-04-20,-
3,00037571,U70101MH1997PTC109823,2011-06-10,-
4,00211074,U67190KA2012PTC063050,2012-03-15,-


In [5]:
# checking which companies exist in the database 
companiesNotInDatabase = []
count = 0 
query = "match (n:company) where n.cin = '{}' return n.name"
for i,cin in enumerate(newLinks['cin']):
    data = graph.data(query.format(cin))
    if len(data) == 0:
        count += 1
        companiesNotInDatabase.append(newLinks.iloc[i][1])
        print('CIN', newLinks.iloc[i][1])

CIN F03989
CIN F01756
CIN F02969
CIN F01756
CIN F01756
CIN F02969
CIN F01756
CIN F01756
CIN F01756
CIN F01756
CIN F01756
CIN F01756
CIN F01756
CIN F03190
CIN F02969
CIN F04070
CIN F02969
CIN F03801
CIN F04070
CIN F03190
CIN F02969
CIN F03295
CIN F03989
CIN F01756


In [6]:
print('Total number of  links to be added = {}'.format(newLinks.shape[0]))
print('Total relations in which companies are not in database : ',len(companiesNotInDatabase))
uniqueCompanies = list(set(companiesNotInDatabase))
print('Distinct companies to be added to database : ', len(uniqueCompanies))

Total number of  links to be added = 230250
Total relations in which companies are not in database :  24
Distinct companies to be added to database :  7


In [7]:
df = pd.DataFrame({'cin':uniqueCompanies})
df.to_csv('companiesNotInDatabase.csv')

In [8]:
# getting max relation ID 
maxRelIdQuery = 'match () -[r]-> () return max(r.relid)'
result = graph.data(maxRelIdQuery)

In [9]:
maxRelId = result[0]['max(r.relid)']

In [10]:
print(maxRelId)

497106


In [11]:
import calendar
import time
import json
checkBusinessperson = "match (b:businessperson) where b.din = '{}' return b"
checkCompany = "match (c:company) where c.cin = '{}' return c"
linkQuery = "match (b:businessperson), (c:company) where b.din='{}' and c.cin='{}' merge (b)-[r:worksin]-> (c) on create set r= {{props}} return r.relid as x"
checkQuery = "match (b:businessperson) -[r:worksin]-> (c:company) where b.din = '{}' and c.cin = '{}' return r"
removeQuery = "match (b:businessperson) -[r:worksin]-> (c:company) where b.din='{}' and c.cin='{}' and r.relid > {} delete r"
newLinks['din'] = newLinks['din'].apply(lambda x: str(x).zfill(8))
relId = maxRelId + 1
c = 0
for i in range(newLinks.shape[0]):
    res = graph.data(checkQuery.format(newLinks.iloc[i]['din'], newLinks.iloc[i]['cin']))
    if len(res) > 0:
        #print(i,'Already Exists')
        c += 1
        continue
    params = {}
    props = {}
    props['relid'] = relId
    props['designation'] = 'Director'
    props['bidirectional'] = 'false'
    props['startdate'] = str(calendar.timegm(time.strptime(newLinks.iloc[i]['startdate'], '%Y-%m-%d')))
    if newLinks.iloc[i]['enddate'] == '-':
        props['iscurrent'] = 'true'
        props['enddate'] = '0'
    else:
        props['iscurrent'] = 'false'
        props['enddate'] = str(calendar.timegm(time.strptime(newLinks.iloc[i]['enddate'], '%d/%m/%Y')))
    params['props'] = props
    res = graph.data(checkBusinessperson.format(newLinks.iloc[i]['din']))
    if len(res) == 0:
        print(newLinks.iloc[i]['din'] , ' business person does not exist')
        continue
    res = graph.data(checkCompany.format(newLinks.iloc[i]['cin']))
    if len(res) == 0:
        print(newLinks.iloc[i]['cin'] , ' company does not exist')
        continue
    #print(graph.data(removeQuery.format(newLinks.iloc[i][1], newLinks.iloc[i][2], maxRelId)))
    res = graph.data(linkQuery.format(newLinks.iloc[i]['din'], newLinks.iloc[i]['cin']), params)
    if len(res) > 0:
        relId += 1
print(c)

F03989  company does not exist
F01756  company does not exist
F02969  company does not exist
F01756  company does not exist
F01756  company does not exist
F02969  company does not exist
F01756  company does not exist
F01756  company does not exist
F01756  company does not exist
F01756  company does not exist
F01756  company does not exist
F01756  company does not exist
F01756  company does not exist
F03190  company does not exist
F02969  company does not exist
F04070  company does not exist
F02969  company does not exist
F03801  company does not exist
F04070  company does not exist
F03190  company does not exist
F02969  company does not exist
F03295  company does not exist
F03989  company does not exist
F01756  company does not exist
122020


In [12]:
print('Already existing relations ', c)
print('Total relations to push ', newLinks.shape[0])

Already existing relations  122020
Total relations to push  230250


In [13]:
# getting max relation ID 
maxRelIdQuery = 'match () -[r]-> () return max(r.relid)'
maxRelIdAfterPush = graph.data(maxRelIdQuery)[0].get('max(r.relid)')
print('Relations pushed = ', maxRelIdAfterPush - maxRelId)

Relations pushed =  108206


In [14]:
checkQuery = 'match (n:businessperson) -[r:worksin] -> (c) where r.relid = {} return n.name, c.name'
for i in range(maxRelId+1, maxRelId+101):
    print(i, graph.data(checkQuery.format(i)))

497107 [{'n.name': 'SHIVAJI TUKARAM AKHADE', 'c.name': 'VIMAL EXTRUSIONS PRIVATE LIMITED'}]
497108 [{'n.name': 'SANDIP KUMAR SINGH', 'c.name': 'FLYTECH CREATIONS PRIVATE LIMITED'}]
497109 [{'n.name': 'MANISH VIMALKUMAR JAIN', 'c.name': 'TRISMA CONSTRUCTIONS PRIVATE LIMITED'}]
497110 [{'n.name': 'SUBBA RAO TELIDEVARA', 'c.name': 'LOTUSPOOL CAPITAL ADVISORS PRIVATE LIMITED'}]
497111 [{'n.name': 'VIKAS BAJAJ', 'c.name': 'ASSOCIATION OF INDIAN FORGING INDUSTRY'}]
497112 [{'n.name': 'RAMA MURTHY BOLLINA', 'c.name': 'GRM PROJECTS & CONSTRUCTIONS PRIVATE LIMITED'}]
497113 [{'n.name': 'LEENA VIPUL MODI', 'c.name': 'JIMEET DEVELOPERS PRIVATE LIMITED'}]
497114 [{'n.name': 'KIRTI KUMAR DOSHI', 'c.name': 'DOSHI CONSULTANTS PRIVATE LIMITED'}]
497115 [{'n.name': 'ADABALA SRIDHAR', 'c.name': 'SRI SRINIVASA AQUA FEEDS PRIVATE LIMITED'}]
497116 [{'n.name': 'PRADEEP KUMAR JINDAL', 'c.name': 'TRUE FAB ENGINEERS PRIVATE LIMITED'}]
497117 [{'n.name': 'NARINDERPAL ROSHANLAL GUPTA', 'c.name': 'HOUSING PROJEC

497196 [{'n.name': 'SURESHKUMAR SHASTRY VEMURI', 'c.name': 'INTEGRATED THERMOPLASTICS LIMITED'}]
497197 [{'n.name': 'SEEMA GUPTA', 'c.name': 'R D P FINANCE PRIVATE LIMITED'}]
497198 [{'n.name': 'RAJ KUMAR SHARMA', 'c.name': 'BRIGHTFUL RETAILERS PRIVATE LIMITED'}]
497199 [{'n.name': 'PRADEEP KUMAR GARG', 'c.name': 'ASTHA TRADELINK LIMITED'}]
497200 [{'n.name': 'ANAND AGARWAL', 'c.name': 'HR EXPLORATION PRIVATE LIMITED'}]
497201 [{'n.name': 'YARLAGADDA PADMANABHA PRASAD SRIMANTHRAJA', 'c.name': 'STARSCAPE COMMUNICATIONS PRIVATE LIMITED'}]
497202 [{'n.name': 'VIJAY JAMNOMAL SHEWAKRAMANI', 'c.name': 'R P HOTELS AND RESTAURANTS PVT LTD'}]
497203 [{'n.name': 'SHARAT CHANDRA BOLUSANI', 'c.name': 'ADVANTA LIMITED'}]
497204 [{'n.name': 'SANDEEP KOHLI', 'c.name': 'THE INDIAN SOCIETY OF ADVERTISERS'}]
497205 [{'n.name': 'HARI SHANKAR BHARTIA', 'c.name': 'JUBILANT OFFSHORE PRIVATE LIMITED'}]
497206 [{'n.name': 'SMITA JAIN', 'c.name': 'LMC ENTERPRISES PRIVATE LIMITED'}]
